A short description on the design of the Kalman functions to build your own dynamic linear model

- library Kalman_LLT for the local linear trend model

- library Kalman_RW for the random walk model

The libraries are composed of 3 functions.

Exemple Kalman_LLT library:

- Kalman_LLT function that is called to estimate the model
- ml_FWD function: runs the Kalman filter with the optimized parameters
- Kalman Filter: runs the Kalman filter and estimate the log-likelyhood function 

The model has to be designed in the Kalman_LLT and in the ml_FWD functions

1. Kalman_LLT(Y,X,sr1,sr2)

Input parameters

- Y: the measurement
- X: explanatory factors
- sr1, sr2: the signal to noise ratios

Output:

- filtered states (spread and slope)
- smoothed states (spread and slope)

Model design:

- nZ: number of states / explanatory factors
- param: vector containing the parameters to be estimated
- A: transition matrix
- Q: covariance matrix of the state innovation error s
- H: covariance matrix
- X0: initial state vector
- P0: initial state covariance matrix. Diagonal.

The parameters to be estimated are:
- the initial state values in X0
- the initial state covariance matrix on the diagonal of P0
- the variance of the measurement error in H

Note, to optimize the convergence of the optimization algorythm (BFGS), the variance parameters are expressed as exp(param), with insure the positivity.

In [ ]:
def Kalman_LLT(y,z,sr1,sr2):
    global _Q, _H, _Y, _Z, _X0, _P0, _A, _nZ, _sr1, _sr2
    Y=y
    Z=z
    nZ=len(Z.T)
    param=np.ones((4+1))
    #INITIAL STATES
    X0=np.zeros((nZ,1))
    #INITIAL STATE COVARIANCE
    P0=np.eye(nZ)*10**6
    #TRANSITION MATRIX
    A=np.eye(nZ)
    A[0,1]=1
    #STATE INNOVATION COVARIANCE MATRIX
    Q=np.zeros((nZ,nZ))  
    #SIGNAL COVARIANCE MATRIX
    H=np.zeros((1,1))    
    #GLOBAL PARAMETERS
    _Q=Q; _H=H; _Y=Y; _Z=Z; _X0=X0; _P0=P0; _A=A; _nZ=nZ; _sr1=sr1; _sr2=sr2
    # OPTIMIZATION
    out_opt=sp.optimize.minimize(ml_FWD,param,method='BFGS', tol=0.001)
    param=out_opt['x']
    print(out_opt['success'])
    #ESTIMATION WITH OPTIMIZED PARAMETERS
    param=param.reshape(max(param.shape))
    H=np.exp(param[0])
    X0=param[1:3]
    Q[0,0]=sr1*H
    Q[1,1]=sr2*H
    P0[0,0]=np.exp(param[3])
    P0[1,1]=np.exp(param[4])
    (likely,XOUT,POUT,XSOUT)=kalman_filter(_Y,_Z,X0,P0,A,Q,H)   
    return XOUT, XSOUT

2. ml_FWD(param)

Input parameters

- param: the parameter vector to be estimated

Output:

- the log-likelyhood function

Model design: define the global matrices/verctors/variables that contain the parameters to be estimated.

In this model:

- _A: transition matrix
- _Q: covariance matrix of the state innovation error s
- _H: covariance matrix
- _X0: initial state vector
- _P0: initial state covariance matrix. Diagonal.

Note, to optimize the convergence of the optimization algorythm (BFGS), the variance parameters are expressed as exp(param), with insure the positivity.

In [ ]:
def ml_FWD(param):
    param=param.reshape(max(param.shape))
    _H=np.exp(param[0])
    _X0=param[1:3]
    _Q[0,0]=_sr1*_H
    _Q[1,1]=_sr2*_H
    _P0[0,0]=np.exp(param[3])
    _P0[1,1]=np.exp(param[4])
    (likely,XOUT,POUT,XSOUT)=kalman_filter(_Y,_Z,_X0,_P0,_A,_Q,_H)
    return -likely


3. Kalman_filter(Y,Z,X0,P0,A,Q,H)

Input parameters

- (Y,Z,X0,P0,A,Q,H) described above

Output:

- log-likelyhood funtion: likely/T
- filtered state vectors: Xout
- state covariance matrix: Pout
- smoothed states vectors: XoutS

In [ ]:
def kalman_filter(Y,Z,X0,P0,A,Q,H):
    #Y(T,1) Z(T,nZ) X0(nZ) P0(nZ,nZ) A(nZ,nZ) Q(nZ,nZ) H(1)   
    nZ=len(Z.T)
    nT=len(Y)
    #FOR FILTER
    Zt=np.zeros((1,nZ))
    Xout=np.zeros((nT,nZ))
    Pout=np.zeros((nT,nZ,nZ))
    #FOR SMOOTHER
    XoutS=np.zeros((nT,nZ))
    XtS=np.zeros((nT,nZ))
    Pt1t=np.zeros((nT,nZ,nZ))
    Ptt=np.zeros((nT,nZ,nZ))
    Xtt=np.zeros((nT,nZ))
    Xt1t=np.zeros((nT,nZ))
    #FILTER
    l1=0.0
    l2=0.0
    likely=0.0
    Xt=X0
    Pt=P0
    #FILTER
    for t in range(0,nT,1):
        #estimation step
        Xte=A.dot(Xt)
        Pte=A.dot(Pt).dot(A.T)+Q            
        #update
        Zt[:,:]=Z[t,:]
        vt=Y[t]-Zt.dot(Xte)
        Ft=Zt.dot(Pte).dot(Zt.T)+H
        if np.linalg.det(Ft)==0:
            Ft=Ft+np.random.normal(0,10**-5,Ft.shape)
        Fti=np.linalg.inv(Ft)
        Xt=Xte+Pte.dot(Zt.T).dot(Fti).dot(vt)
        Pt=(np.eye(nZ)-Pte.dot(Zt.T).dot(Fti).dot(Zt)).dot(Pte)
        #stockage
        Xout[t,:]=Xt.T
        Pout[t,:,:]=Pt
        l1=l1+np.log(np.linalg.det(Ft))
        l2=l2+(vt.T).dot(Fti).dot(vt)
        #pour smoother
        Ptt[t,:,:]=Pt
        Xtt[t,:]=Xt.T
        Pt1t[t,:,:]=Pte
        Xt1t[t,:]=Xte.T
    likely=-0.5*nT*np.log(2*math.pi)-0.5*l1-0.5*l2
    #SMOOTHER
    XtS[nT-1,:]=Xtt[nT-1,:]
    XoutS[nT-1,:]=Xtt[nT-1,:]
    for t in range(nT-2,-1,-1):
        if abs(np.linalg.det(Pt1t[t+1,:,:]))==0:
            Pt1t[t+1,:,:]=Pt1t[t+1,:,:]+np.random.normal(0,10**-5,Pt1t[t+1,:,:].shape)
        inverse=np.linalg.inv(Pt1t[t+1,:,:])
        Ft=Ptt[t,:,:].dot(A.T).dot(inverse)
        XtS[t,:]=Xtt[t,:]+Ft.dot(XtS[t+1,:]-Xt1t[t+1,:])
        XoutS[t,:]=XtS[t,:]
    likely=np.reshape(likely,(1))
    return likely/nT,Xout,Pout,XoutS